In [93]:
import pandas as pd
from bs4 import BeautifulSoup

---

## 1. Price parsing

- `original_price`: Giá gốc (giá chưa giảm, đơn vị: VND)
- `sale_price`: Giá khuyến mãi (giá đã giảm, đơn vị: VND)
- `precent_discount`: Phần trăm số tiền đã giảm (đơn vị: %)

<pre>
- Có giá khuyến mãi: 
    &lt;fare-sale&gt; chứa giá khuyến mãi
        &lt;fareSmall&gt; chứa giá gốc và phần trăm ưu đãi
- Không có giá khuyến mãi:
    &lt;fare&gt; chứa giá gốc
        &lt;fareSmall&gt; không chứa dữ liệu
</pre>

In [94]:
def has_no_discount_price(block) -> bool:
    '''
    Phân loại chuyến xe này có giá khuyến mãi hay không \n
    True: không có giá khuyến mãi \n
    False: có giá khuyến mãi
    '''
    fare = block.find('div', class_='fare')      
    
    if fare: 
        return True  # khong co gia khuyen mai
    
    return False     # co gia khuyen mai

def parse_fare(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi không có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi = None, phần trăm khuyến mãi = None
    '''

    sale_price = None
    percent_discount = None

    fare = block.find('div', class_='fare')
    original_price = fare.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()    
    
    return {
        "original_price": original_price ,
        "sale_price":sale_price,
        "percent_discount": percent_discount
        }

def parse_fare_small(block):
    '''
    Dành cho trường hợp có khuyến mãi \n
    Trả về: giá gốc, phần trăm khuyến mãi
    '''

    fare_small = block.find('div', class_='fareSmall')

    original_price = fare_small.find('div', class_='small').get_text(strip=True).replace("đ", "").strip()
    if fare_small.find('div', class_='percent'):
        percent_discount = fare_small.find('div', class_='percent').get_text(strip=True)
    else:
        percent_discount = None
        
    return original_price, percent_discount

def parse_fare_sale(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi, phần trăm khuyến mãi
    '''

    fare_sale = block.find('div', class_='fare-sale')
    original_price, percent_discount = parse_fare_small(block)
    
    if fare_sale and fare_sale.get_text(strip=True):
        sale_price = fare_sale.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
        
    return {
        "original_price": original_price ,
        "sale_price":sale_price,
        "percent_discount": percent_discount
        }

In [95]:
def parse_price(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe \n
    Trả về: giá gốc, giá khuyến mãi (nếu có), phần trăm khuyến mãi (nếu có)
    '''
    
    if has_no_discount_price(block):
        return parse_fare(block)
    else:
        return parse_fare_sale(block)

---

## 2. Bus info parsing

In [96]:
def parser_trip_bus_info(container):
    '''
    Trích xuất thông tin từ một container chứa thông tin chuyến đi. \n
    Trả về Tuple: tên nhà xe, đánh giá nhà xe, loại ghế.
    '''
    
    # bus name / company name
    bus_element = container.find('div', class_='bus-name')
    bus_name = bus_element.get_text(strip=True) if bus_element else None

    # bus rating
    rating_element = container.find('div', class_='bus-rating').find('span')
    bus_rating = rating_element.get_text(strip=True) if rating_element else None

    # seat_type
    seat_type = container.find('div', class_='seat-type')
    seat_type = seat_type.get_text(strip=True) if seat_type else None

    return {
        'bus_name': bus_name,
        'bus_rating': bus_rating,
        'seat_type': seat_type
    }

---

## 3. Route parsing

In [113]:
# Dữ liệu này nằm ở ô filter chuyến đi

def parse_route_info(block):
    '''
    Trích xuất dữ liệu từ filter của trang web \n
    Trả về: ngày khởi hành, nơi xuất phát (thành phố hiện tại), nơi đến (nơi đặt vé đến)
    '''

    departure_date, start_point, destination = None, None, None

    try:
        departure_date = block.find('p', class_='date-input-value').get_text(strip=True)
        start_point = block.find(id="from_input").get('value')
        destination = block.find(id="to_input").get('value')
    except Exception:
        pass

    return {
        'departure_date': departure_date,
        'start_point': start_point,
        'destination': destination
    }

---

## 4. Details trip info

### 4.1 Departure

In [98]:
# Dữ liệu này nằm trong container > 'from_content'

def parse_departure_trip_info(from_content):
    """
    Trích xuất thông tin điểm đi từ một container 'from_content'.
    Trả về một tuple chứa: giờ khởi hành, địa điểm đón khách.
    """
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not from_content:
        return None, None

    # departure time
    from_hour_tag = from_content.find('div', class_='hour')
    departure_time = from_hour_tag.get_text(strip=True) if from_hour_tag else None

    # departure place
    from_place_tag = from_content.find('div', class_='place')
    pick_up_point = from_place_tag.get_text(strip=True) if from_place_tag else None
    
    return {
        'departure_time': departure_time,
        'pick_up_point': pick_up_point
    }

### 4.2 Arrival

In [99]:
def parse_arrival_trip_info(to_content):
    """
    Trích xuất thông tin điểm đến từ một container 'to_content'.\n
    Trả về một tuple chứa: ngày đến, thời gian đến, điểm trả khách.
    """
    
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not to_content:
        return None, None, None

    # lấy ngày đến
    date_arrival_tag = to_content.find('span', class_="text-date-arrival-time")
    arrival_date = date_arrival_tag.get_text(strip=True) if date_arrival_tag else None
    
    
    # lấy giờ và địa điểm trả khách
    content_to_info = to_content.find('div', class_='content-to-info')
    if content_to_info:
        to_hour_tag = content_to_info.find('div', class_='hour')
        arrival_time = to_hour_tag.get_text(strip=True) if to_hour_tag else None
        
        to_place_tag = content_to_info.find('div', class_='place')
        drop_off_point = to_place_tag.get_text(strip=True) if to_place_tag else None
        
    return {
        'arrival_date': arrival_date,
        'arrival_time': arrival_time,
        'drop_of_point': drop_off_point
    }


---

In [100]:
def get_departure_arrival_trip(container):
    """
    Hàm chính để trích xuất toàn bộ chi tiết chuyến xe từ một container lớn.
    """
    
    # tìm container cha chứa cả from và to
    from_to_content = container.find('div', class_="from-to-content")

    # nếu không có container, không có thông tin gì để xử lý
    if not from_to_content:
        return {
            "duration": None,
            "from_hour": None,
            "from_place": None,
            "departure_date": None,
            "arrival_date": None,
            "to_hour": None,
            "to_place": None,
        }

    # departure element and information 
    from_content = from_to_content.find('div', class_='content from')
    dict_departure_info = parse_departure_trip_info(from_content)

    # arrival element and information
    to_content = from_to_content.find('div', class_='content to')
    dict_arrival_info = parse_arrival_trip_info(to_content)

    # duration for trip
    duration_tag = from_to_content.find('div', class_="duration")
    duration = duration_tag.get_text(strip=True) if duration_tag else None

    trip_data = dict_departure_info | dict_arrival_info | {'duartion': duration}
    
    return trip_data

In [101]:
def gather_trip_info(block):
    '''
    Hàm chính để tập hợp toàn bộ thông tin chuyến xe.
    '''

    # main bus info
    dict_bus_info = parser_trip_bus_info(block)
    
    # trip details
    dict_trip_details = get_departure_arrival_trip(block)
    
    # price details
    dict_price= parse_price(block)
    
    # combine all data into a single dictionary
    trip_data = dict_bus_info | dict_trip_details | dict_price
    
    return trip_data

In [102]:
# def parse_bus_info(container):

# # -------------------------------------TIME---------------------------------------------------------------
#     from_to_content = container.find('div', class_="from-to-content")
#     if from_to_content:
#         to_content = from_to_content.find('div', class_='content to')
#         from_content = from_to_content.find('div', class_='content from')
#         duration = from_to_content.find('div', class_="duration").get_text(strip=True) if from_to_content.find('div', class_="duration") else None

#         # Arrival info
#         date_arrival = None
#         to_hour = None
#         to_place = None
#         if to_content:
#             span = to_content.find('span', class_="text-date-arrival-time")
#             date_arrival = span.get_text(strip=True) if span else None
#             content_to_info = to_content.find('div', class_='content-to-info')
#             if content_to_info:
#                 to_hour = content_to_info.find('div',class_='hour' ).get_text(strip=True) if content_to_info.find('div',class_='hour' ) else None
#                 to_place = content_to_info.find('div',class_='place' ).get_text(strip=True) if content_to_info.find('div',class_='place' ) else None

#         # Departure info
#         from_hour = from_content.find('div',class_='hour' ).get_text(strip=True) if from_content and from_content.find('div',class_='hour' ) else None
#         from_place = from_content.find('div',class_='place' ).get_text(strip=True) if from_content and from_content.find('div',class_='place' ) else None
#     else:
#         duration = None
#         date_arrival = None
#         to_hour = None
#         to_place = None
#         from_hour = None
#         from_place = None


# # --------------------------------------PRICE--------------------------------------------------------------
#     price_original, price_discount, discount_percent = parse_price(container)

    

#     return [
#         bus_name, bus_rating, seat_type,
#         from_hour, from_place, duration,
#         to_hour, to_place, date_arrival,
#         price_original, price_discount, discount_percent, notification
#     ]

---

rating

In [103]:
def extract_rating_from_container(container):
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)
                rate_point = rate_ps[1].get_text(strip=True)
                ratings.append((rate_title, rate_point))
        if ratings:
            return ratings
        else:
            return [(None, None)]
    except Exception:
        return [(None, None)]

total

In [116]:
# Gộp dữ liệu chuyến xe và các đánh giá lại với nhau
def get_all_bus_trip_info(soup):
    '''
    Kết hợp các dữ liệu lại với nhau. \n
    Trả về 1 DataFrame
    '''

    dict_route = parse_route_info(soup)

    containers = soup.find_all("div", class_="container")

    lst_trips_info = []

    for container in containers:
        # 
        dict_trip_info = gather_trip_info(container) | dict_route   # dict
        df_trip_info = pd.DataFrame([dict_trip_info])  # 1 hàng

        # lấy dữ liệu rating
        lst_detail_rating = extract_rating_from_container(container)  # list
        df_ratings = pd.DataFrame([dict(lst_detail_rating)])  # Chuyển list -> dict -> df 1 hàng

        # gộp 2 df lại
        df = pd.concat([df_trip_info, df_ratings], axis=1)

        lst_trips_info.append(df)

    all_trips_info = pd.concat(lst_trips_info, ignore_index=True)

    return all_trips_info

In [105]:
with open("../../data/site/site_updated_parse_bus_info.html", "r", encoding="utf-8") as file:
    content = file.read()
soup = BeautifulSoup(content, "html.parser")

---

In [106]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

### Click button 'read more'

In [107]:
def click_load_more(driver):
    while True:
        try:
            load_more_span = driver.find_element(By.XPATH, "//span[text()='Xem thêm chuyến']")
            load_more_button = load_more_span.find_element(By.XPATH, "./ancestor::button")
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()
            time.sleep(random.uniform(2, 3))
        except NoSuchElementException:
            break
        except ElementClickInterceptedException:
            time.sleep(2)

# FLOW OFFICIAL

---

## Crawl rating data

# -- Main --

In [108]:
# Click 'xem thêm chuyến' đến hết -> hiển thị tất cả các chuyến xe
# click_load_more(driver)

# Open all ratings after all trips are loaded
# try:
#     stars = WebDriverWait(driver, 15).until(
#         EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".bus-rating-button .anticon-star"))
#     )
#     for star in stars:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
#         time.sleep(1.5)
#         ActionChains(driver).move_to_element(star).click().perform()
#         time.sleep(1)
# except Exception as e:
#     print("Error clicking star icons:", e)

In [109]:
import requests
soup = BeautifulSoup(requests.get('https://vexere.com/vi-VN/ve-xe-khach-tu-sa-pa-lao-cai-di-quang-ninh-2424t1491.html?date=11-10-2025&v=6&nation=84').content, 'html.parser')

In [110]:
with open('../../data/site/new_crawl_data.html', 'r', encoding='utf-8') as file:
    data = file.read()

In [111]:
soup = BeautifulSoup(data, 'html.parser')

In [117]:

# driver = webdriver.Chrome()
# driver.get("https://vexere.com/vi-VN/ve-xe-khach-tu-sai-gon-di-nha-trang-khanh-hoa-129t23591.html?date=11-10-2025&v=6")

# Click 'xem thêm chuyến' đến hết -> hiển thị tất cả các chuyến xe
# click_load_more(driver)

# Open all ratings after all trips are loaded
# try:
#     stars = WebDriverWait(driver, 15).until(
#         EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".bus-rating-button .anticon-star"))
#     )
#     for star in stars:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
#         time.sleep(1.5)
#         ActionChains(driver).move_to_element(star).click().perform()
#         time.sleep(1)
# except Exception as e:
#     print("Error clicking star icons:", e)

# time.sleep(1.5)
# soup = BeautifulSoup(driver.page_source, "html.parser")

lst_trips_info = []
df_trips_info = get_all_bus_trip_info(soup)

# df_trips_info.to_csv("../../data/raw/data_updated_parse_bus_info.csv", index=False)
# driver.quit()


In [118]:
df_trips_info

,bus_name,bus_rating,seat_type,departure_time,pick_up_point,arrival_date,arrival_time,drop_of_point,duartion,original_price,...,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,None
0,Đà Lạt ơi,4.8 (3425),Limousine 24 Phòng ĐÔI,23:30,• Trạm Hàng Xanh,(12/10),06:30,• Trạm Nha Trang,7h,450.000,...,Sài Gòn,Nha Trang - Khánh Hòa,4.8,4.8,4.8,4.8,4.8,4.8,4.8,NaN
1,Bình Minh Tải,4.7 (3650),Limousine 22 Phòng Đơn,22:30,• Văn Phòng Quận 1,(12/10),05:35,• Văn phòng Nha Trang,7h5m,350.000,...,Sài Gòn,Nha Trang - Khánh Hòa,4.6,4.6,4.7,4.6,4.6,4.6,4.8,NaN
2,Huỳnh Gia,4.7 (8534),Giường nằm 38 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,(12/10),05:00,• Văn Phòng Nha Trang,6h30m,280.000,...,Sài Gòn,Nha Trang - Khánh Hòa,4.8,4.7,4.7,4.7,4.6,4.6,4.7,NaN
3,An Anh Limousine,4.8 (8322),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,(12/10),06:00,• Văn phòng Nha Trang,6h30m,299.000,...,Sài Gòn,Nha Trang - Khánh Hòa,4.8,4.7,4.7,4.7,4.7,4.7,4.6,NaN
4,Khanh Phong,4.7 (16845),Limousine 20 giường phòng (WC),05:15,• Văn Phòng Phạm Ngũ Lão - Quận 1.,None,11:25,• Văn Phòng Nha Trang (KS Mường Thanh),6h10m,480.000,...,Sài Gòn,Nha Trang - Khánh Hòa,4.8,4.7,4.7,4.7,4.6,4.6,4.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:30,• Văn Phòng Sài Gòn,(12/10),06:10,• Chợ Vĩnh Lương,7h40m,480.000,...,Sài Gòn,Nha Trang - Khánh Hòa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
561,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,(12/10),06:00,• Văn Phòng Nha Trang,7h15m,470.000,...,Sài Gòn,Nha Trang - Khánh Hòa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
562,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,(12/10),06:10,• Chợ Vĩnh Lương,7h25m,480.000,...,Sài Gòn,Nha Trang - Khánh Hòa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
563,Quỳnh Nhật,3.7 (279),Limousine 24 phòng,10:00,• Bến xe Miền Đông mới,None,18:05,• Ngã 3 Thành.,8h5m,650.000,...,Sài Gòn,Nha Trang - Khánh Hòa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [119]:
df_trips_info.shape

(565, 23)

In [ ]:
# with open("../../data/site.site_updated_parse_bus_info.html", "w", encoding="utf-8") as file:
#     file.write(driver.page_source)